In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from pyadlml.constants import *
import matplotlib.pyplot as plt

#### Not alternating PE

In [ ]:
from pyadlml.preprocessing import PositionalEncoding

X = np.arange(0, 100)
pe = PositionalEncoding(d_dim=50, inplace=False)
pe.fit()
print(pe.max_period, pe.min_freq)
ma = pe.transform(X)
pe.plot_angular_freq()
pe.plot_pe_mat(X)
pe.plot_dotproduct_similarity(X, 50);

In [ ]:
from pyadlml.preprocessing import TimePositionalEncoding
 
    
X = pd.DataFrame(columns=[TIME], 
                  data=pd.date_range(start='6/1/2020', end='6/2/2020 00:30:00')[:-1])
X['f1'] = 2

pe = TimePositionalEncoding(d_dim=20, inplace=False, res=pd.Timedelta('1ms'))
pe.fit()
print(pe.res, pe.max_period, pe.min_freq)
#ma = pe.transform(X)
pe.plot_angular_freq()
pe.plot_waves(1e6)
pe.plot_pe_mat(X)
pe.plot_dotproduct_similarity(X, 50)
;

In [ ]:
from pyadlml.preprocessing import CyclicTimePositionalEncoding
    
    
X = pd.DataFrame(columns=[TIME], 
                  data=pd.date_range(start='6/1/2020', end='6/2/2020 00:30:00', freq='1s')[:-1])
X['f1'] = 2
# 1.3
pe = CyclicTimePositionalEncoding(d_dim=20, max_period='1D', min_period='1min').fit()
#print(pe.get_periods())
#print(pe.get_max_base())
#pe.plotly_wave_length_per_dim().show()

#print(pe.res, pe.max_period, pe.min_freq)
ma = pe.transform(X)
#pe.plotly_wave_length_per_dim().show()
#pe.plot_angular_freq()
#pe.plot_waves(1e6)
#pe.plot_pe_mat(X)
#pe.plot_dotproduct_similarity(X, 0);
#pe._num2td(pe.get_periods().min())
;

In [ ]:
import numpy as np
import plotly.graph_objects as go

#  Generate x values from 0 to 2*pi
lmbd = 8640

f = 1/lmbd
w = np.pi*2*f
x = np.linspace(0, lmbd+100, lmbd*8)

# Compute y values using the sine function
y = np.sin(w*x)
denom = 2**15
f = 1/(lmbd/min(denom,lmbd))
print(f, denom, lmbd)
w = np.pi*2*f

y1 = np.sin(w*x)
e=1
# Create traces
trace0 = go.Scatter(x=x, y=y, mode='lines', name=f'f={f:.2f}, lmbd={1/f:.2f}, e={e}')
trace1 = go.Scatter(x=x, y=y1, mode='lines', name=f'f={f:.2f}, lmbd={1/f:.2f}, e={e}')
trace2 = go.Scatter(x=[np.pi, np.pi], y=[-1, 1], mode='lines', name='pi line', line=dict(color='black', dash='dot'))
trace3 = go.Scatter(x=[1, 1], y=[-1, 1], mode='lines', name='1 line', line=dict(color='black'))
trace4 = go.Scatter(x=[lmbd, lmbd], y=[-1, 1], mode='lines', name='lmbd line', line=dict(color='red'))

layout = go.Layout(title='Sine Function', 
                   xaxis=dict(title='x'), 
                   yaxis=dict(title='sin(x)'))

fig = go.Figure(data=[
    trace0,
    trace1, 
    trace2,
    trace3,
    trace4
], layout=layout)


# Display the plot
fig.show()



In [ ]:
X.at[0, TIME]

In [ ]:
pd.Timedelta('1ms').total_seconds()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# define the form of the function we want to fit
base = 2.16
#def func(x, a, b):
#    return a * np.power(base, b * x)

# wokring
#def func(x, a):
#    return 4.0*np.power(21600/4.0, (x/15))

d = 15
o = 10
max_period = 21600.0    # 6h
min_period = 4.0        # 4s

def func(i, a, b):
    return (1/max_period)*np.power(np.e, b*i)
def func(i, a,b):
    return 1/(np.round(np.power(max_period/min_period, (-i+(d-1))/(d-1)))/max_period)

# points
xdata = np.array([0, d])
ydata = np.array([min_period, max_period])

# use curve_fit to find optimized values for a and b
popt, pcov = curve_fit(func, xdata, ydata)
print(popt)

# create a range of x values
x = np.linspace(0, d, 400)

# calculate the corresponding y values
y = func(x, *popt)

y_d = func(np.arange(0,d), *popt).astype(int)
# plot the points
plt.scatter(xdata, ydata, color='red')
plt.scatter(np.arange(1, d-1), y_d[1:-1], color='blue')
# plot the fit
plt.plot(x, y, label=f'a')
print(y_d)

plt.legend()
plt.show()
plt.clr()

In [ ]:
import numpy as np
import pandas as pd
from pyadlml.constants import TIME
import plotly.graph_objects as go
import pandas as pd
from pyadlml.constants import TIME

res = '100ms'
min_period = pd.Timedelta('4s')
max_period = pd.Timedelta('6h')
min_period_posx = int(min_period/pd.Timedelta(res))
max_period_posx = int(max_period/pd.Timedelta(res))
X = pd.DataFrame(columns=[TIME], 
                  data=pd.date_range(start='6/1/2020', end='6/1/2020 06:30:00', freq=res)[:-1])
X['f1'] = 2

num_min_period = min_period.total_seconds()
num_max_period = max_period.total_seconds()

print('n minp: ', num_min_period)
print('n maxp: ', num_max_period)

# Convert timestamps to numeric in seconds
td = (X[TIME] - pd.to_datetime(X[TIME].dt.date))
x = td.dt.total_seconds()

# one period in 6h
f2 = 1/num_max_period       
w2 = np.pi*2*f2
y2 = np.sin(w2*x)

# one period in 4s
f3 = 1/num_min_period
w3 = np.pi*2*f3
y3 = np.sin(w3*x)


d = 15
i = 0
#denom = (num_min_period*np.power(num_max_period/num_min_period, i/d))
denom = round(np.power(num_max_period/num_min_period, (-i+d)/d))/num_max_period
f1 = denom
w1 = np.pi*2*f1
y1 = np.sin(w1*x)

denom = 2**15
f4 = 1/(num_max_period/min(denom, num_max_period))
w4 = np.pi*2*f4
y4 = np.sin(w4*x)
e=0


# Create traces
trace1 = go.Scatter(x=X[TIME], y=y1, mode='lines', name=f'1: f={f1:.2f}, lmbd={1/f1:.2f}')
trace2 = go.Scatter(x=X[TIME], y=y2, mode='lines', name=f'2: f={f2:.2f}, lmbd={1/f2:.2f}=maxp')
trace3 = go.Scatter(x=X[TIME], y=y3, mode='lines', name=f'3: f={f3:.2f}, lmbd={1/f3:.2f}=minp')
trace5 = go.Scatter(x=[X.at[min_period_posx, TIME]]*2, y=[-1, 1], mode='lines', name=f'{min_period}', line=dict(color='black'))
trace6 = go.Scatter(x=[X.at[max_period_posx, TIME]]*2, y=[-1, 1], mode='lines', name='line', line=dict(color='black'))

layout = go.Layout(title='Sine Function', 
                   xaxis=dict(title='x'), 
                   yaxis=dict(title='sin(x)'))

fig = go.Figure(data=[
    trace1,
    trace2,
    trace3,
    trace5,
    trace6,
], layout=layout)

fig.update_layout(
    xaxis_range=[X.at[0, TIME], X.at[min_period_posx, TIME]],
    yaxis=dict(fixedrange=True)
)
# Display the plot
fig.show()

In [2]:
from pyadlml.preprocessing import CyclicTimePositionalEncoding
from pyadlml.constants import TIME
import pandas as pd
 
#ctpe = CyclicTimePositionalEncoding(d_dim=16, max_period='6h', min_period='4s').fit()
#ctpe.plotly_waves().show()
X = pd.DataFrame(columns=[TIME], 
                  data=pd.date_range(start='6/2/2020', end='6/2/2020 00:30:00', freq='10s'))
X['f1'] = 2

pe = CyclicTimePositionalEncoding(d_dim=16, max_period='24h', min_period='4s')
pe.fit()
#ma = pe.transform(X)
#pe.plot_angular_freq()
#pe.plot_pe_mat(X)
#pe.plot_dotproduct_similarity(X, 170);
# #pe.plotly_wave_length_per_dim()

CyclicTimePositionalEncoding(d_dim=16, max_period=Timedelta('1 days 00:00:00'),
                             min_period=Timedelta('0 days 00:00:04'))

In [18]:
def plotly_wave_length_per_dim(self):
    from plotly import graph_objects as go
    import numpy as np
    fig = go.Figure()
    ws = self.get_angular_freqs()
    f = ws/(2*np.pi)
    lmbd = 1/f
    periods = self._num2td(lmbd)
    print(periods.dtype)
    x = pd.Timestamp('01.01.2000 00:00:00') + periods
    fig.add_trace(
        go.Scatter(
            y=np.arange(0, self.d_dim), 
            x=x,
            customdata=pd.to_timedelta(periods).map(str),
            hovertemplate="Dim: %{y}<br>Period: %{customdata}<br><extra></extra>"
        )
    )
    fig.update_layout(title='Period per dim')
    return fig

plotly_wave_length_per_dim(pe)

timedelta64[ns]


In [ ]:
ctpe.get_angular_freqs()

In [ ]:
class TimePositionalEncoding(PositionalEncoding):

    def __init__(self, d_dim, max_period='6h', min_period='1s', inplace=True):
        super().__init__(d_dim, max_period)  # Add super call to the parent class
        self.min_period = min_period
        self.inplace = inplace

    def fit(self, X=None, y=None):
        # Map timestamp values to a sequence
        self.min_period = self._td2num(pd.Timedelta(self.min_period))
        self.max_period = self._td2num(pd.Timedelta(self.max_period))
        return self

    def _td2num(self, x):
        if not isinstance(x, np.ndarray) and not isinstance(x, pd.Series):
            x = pd.Timedelta(x)
        else:
            return x.dt.total_seconds()

        return x.total_seconds()

    def _num2td(self, x):
        return pd.Timedelta('1s')*x

    def get_angular_freqs(self):
        """period of length 1"""
        i = np.arange(self.d_dim)//2
        f = self.min_period*np.power(self.max_period/self.min_period, (2*i/self.d_dim))
        ws = 2*np.pi*f
        return ws

    def transform(self, X, y=None, inplace=None):
        """ 

        sin(w*t) 
        """
        inplace = self.inplace if inplace is None else inplace

        if isinstance(X, pd.DataFrame):
            x = X[TIME]
        assert isinstance(x, pd.Series), 'TimePosEnc: Sth. went wrong'
        # Convert timestamps to numeric in seconds
        td = (x - pd.to_datetime(x.dt.date))
        t = self._td2num(td).to_numpy()
        freqs = self.get_angular_freqs()

        # calc pos enc (T, d_dim)
        xt = self._pos_enc(freqs, t)
        if inplace:
            xt = pd.DataFrame(data=xt, columns=[
                              f'pe_dim_{i}' for i in range(0, self.d_dim)])
            xt = pd.concat([X, xt], axis=1)

        return xt


 
    


In [ ]:
pd.Timedelta('24h')/pd.Timedelta('1000ms')

In [ ]:
np.log2(8640)

In [ ]:
d_model = 32
seq_length = 864000
max_period = 864000
min_freq = 1/max_period
#min_freq = 1e-3
#min_freq = 100
print(d_model, max_period, min_freq)

In [ ]:
d_model = 32
seq_length = 864000
max_period = 10000
min_freq = 1/max_period
#min_freq = 1e-3
#min_freq = 100
print(d_model, max_period, min_freq)

In [ ]:
def w_discon(d_dim, min_freq):
    i = np.arange(d_dim)//2
    ws = np.power(min_freq, (2*i/d_dim))
    return ws
    
def w_discon1(d_dim, min_freq):
    """period of length 1"""
    i = np.arange(d_dim)//2
    ws = (2*np.pi)*np.power(min_freq, (2*i/d_dim))
    return ws

def w_discon2(d_dim, min_freq):
    """Take only every second angular frequency"""
    d_dim = d_dim*2
    i = np.arange(d_dim)//2
    ws = np.power(min_freq, (2*i/d_dim))
    return ws[1::2]

def w_discon3(d_dim, min_freq):
    """period of length 1"""
    b = 2
    i = np.arange(d_dim)//2
    lmbd = 1/min_freq
    tmp = b**(i)
    f = 1/(lmbd/(np.minimum(tmp, lmbd)))
    ws = 2*np.pi*f
    return ws

w = w_discon3
    #d = (n_dim-d)//2
    #lmbda = pd.Timedelta('24h')/min_freq**(2*d)
    #lmbda = td2num(lmbda, res)
    #return 1/lmbda*(2*np.pi)
lmbd = lambda freqs: 2*np.pi*freqs
print('w:\n', w(d_model, min_freq))
print('period:\n', lmbd(w(d_model, min_freq)))

In [ ]:
def plot_angular_freq(w, d_model, min_freq=1e-3):

    n = w(d_model, min_freq=min_freq)
    #plt.yscale('linear')
    plt.plot(np.arange(len(n)), n, label=f'min{min(n)}')
    plt.xlabel('d - dimension')
    plt.ylabel('$\omega(d)$')
    plt.grid(True)
    plt.legend()

plot_angular_freq(w_discon3, d_model, min_freq)
plot_angular_freq(w, d_model, min_freq)


In [ ]:
def plot_waves(freqs, max_n):
    import matplotlib.pyplot as plt

    n_dim = len(freqs)

    # Create a figure and an axis
    fig, ax = plt.subplots(figsize=(10,10))

    # Loop over the range of dimensions
    x = np.linspace(0, max_n, 1000)
    for d in range(n_dim):   
        oscil = np.sin if d%2==0 else np.cos     
        ax.plot(x, oscil(freqs[d]*x)+2*d, label=f'pe dim={d}')
    # Set the axis scales to be equal
    #ax.set_aspect('equal', 'box')

    # Show the legend
    ax.legend()


plot_waves(w(d_dim=d_model, min_freq=min_freq), seq_length)

In [ ]:
w(d_model, min_freq)[::-1]


In [ ]:
def positional_encoding(position, d_model, min_freq=1e-4):
    freqs = w(d_model, min_freq)
    pos_enc = position.reshape(-1,1)*freqs.reshape(1,-1)
    pos_enc[:, ::2] = np.cos(pos_enc[:, ::2])
    pos_enc[:, 1::2] = np.sin(pos_enc[:, 1::2])
    return pos_enc

positional_encoding(np.arange(10), d_model=6, min_freq=1e-1)

In [ ]:
### Plotting ####
max_pos = seq_length
mat = positional_encoding(np.arange(max_pos), d_model, min_freq=min_freq)
plt.pcolormesh(mat, cmap='copper')
plt.xlabel('Depth')
plt.xlim((0, d_model))
plt.ylabel('Position')
plt.title("PE matrix heat map")
plt.colorbar()
plt.show()

In [ ]:
def plot_dotproduct_similarity(pe, pos):
    # let's choose the first vector
    first_vector = pe[pos, :]

    # compute the dot products
    dot_products = np.dot(pe,first_vector)

    # visualize with matplotlib
    plt.figure(figsize=(10, 6))
    plt.plot(dot_products)
    plt.title('Dot products of the first vector with the rest')
    plt.xlabel('Vector index')
    plt.ylabel('Dot product')
    plt.grid(True)
    
plot_dotproduct_similarity(mat, 0)

In [ ]:
def pos_enc(t:np.ndarray, n_dim: int, w): 
    PE = np.zeros((len(t), n_dim))
    angular_freqs = w(n_dim) min_freq**(2*(np.arange(d_model)//2)/d_model)
    for i, t in enumerate(t):
        for d in range(n_dim):
            if d%2 == 0:
                PE[i, d] = np.sin(w(d)*t)
            else:
                PE[i, d] = np.cos(w(d)*t)
    return PE

In [ ]:
df = pd.DataFrame(columns=[TIME], 
                  data=pd.date_range(start='6/1/2020', end='6/3/2020 00:30:00', freq='1s')[:-1])
df['f1'] = 2
df

In [ ]:
pe
sns.set(rc={'figure.figsize':(14,4)})
ax = sns.heatmap(pe.T[:,:])
ax.invert_yaxis()
ax.set_ylabel("dimension d")
ax.set_xlabel("time step")
ax.set_title("Sinusoid absolute positional encoding")

In [ ]:
pe.shape

In [ ]:
def get_suitable_base(d, min_period):
    return (np.log(td2num(pd.Timedelta('24h'), res=min_period)/d)/ np.log(d))*2

[get_suitable_base(i, '10ms') for i in range(1, 20)]

In [ ]:
get_suitable_base(3, '100ms' aaaaaq)

In [ ]:
import numpy as np
import seaborn as sns
def get_position_angle_vec(position, dimensions):
    return [position / np.power(10000, 2 * (i // 2) / (dimensions))
            for i in range(dimensions)]

def sinusoid_positional_encoding_ref(length, dim):   
    PE = np.array([get_position_angle_vec(i, dim) for i in range(length)])
    PE[:, 0::2] = np.sin(PE[:, 0::2])
    PE[:, 1::2] = np.cos(PE[:, 1::2])
    return PE

pe2 = sinusoid_positional_encoding_ref(20000, 40)

sns.set(rc={'figure.figsize':(14,4)})
ax = sns.heatmap(pe2.T)
ax.invert_yaxis()
ax.set_ylabel("dimension d")
ax.set_xlabel("time step")
ax.set_title("Sinusoid absolute positional encoding")